In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [16]:
from dotenv import load_dotenv
import os

# Check if .env file exists
if os.path.exists('.env'):
    load_dotenv() 
    
openai_api_key = os.environ.get("OPENAI_ACCESS_TOKEN")

In [68]:
db = SQLDatabase.from_uri(
    "sqlite:///../data/sqlite_llm_dummy.db",
    sample_rows_in_table_info=1
    )
#print(db.get_table_info())
sql_query = """
SELECT
    xa.name AS account_name,
    SUM(tli.line_amount) AS total_expense
FROM
    xero_accounts xa
JOIN
    transactions t ON xa.account_id = t.account_id
JOIN
    transactions__line_items tli ON t._id = tli._sdc_source_key__id
WHERE
    xa.account_id = 'acc3' -- Provided account_id
    AND t.date >= '2024-03-13 08:00:00' AND t.date <= '2024-03-13 11:00:00' -- Provided date range
    AND t.type = 'Expense' -- Assuming 'Expense' is the type for expense transactions
GROUP BY
    xa.name;

"""
db.run(sql_query)

''

In [57]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

chain = create_sql_query_chain(llm, db)

query = """
what is the total number of transactions?
"""

question = """Can you share the total expenses for a given account_id in a given date range. The output
should include account name and total expense"""

response = chain.invoke({"question": question})
print(response)

SELECT x.name AS "account_name", SUM(t.total) AS "total_expense"
FROM transactions t
JOIN xero_accounts x ON t.account_id = x.account_id
WHERE t.account_id = 'your_account_id_here'
AND t.date BETWEEN 'start_date_here' AND 'end_date_here'
GROUP BY x.name


In [58]:
db.run(response)

''

In [50]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [51]:
agent_executor.invoke(
    "how many transactions are there?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


transactions, transactions__line_items, xero_accounts
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	_id TEXT NOT NULL, 
	_sdc_batched_at TIMESTAMP, 
	_sdc_extracted_at TIMESTAMP, 
	_sdc_received_at TIMESTAMP, 
	_sdc_sequence INTEGER, 
	_sdc_table_version INTEGER, 
	account_id TEXT, 
	bank_account_code TEXT, 
	bank_account_id TEXT, 
	bank_account_name TEXT, 
	bank_transaction_id TEXT, 
	contact_id TEXT, 
	contact_name TEXT, 
	currency_code TEXT, 
	date TIMESTAMP, 
	is_reconciled BOOLEAN, 
	line_amount_types TEXT, 
	reference TEXT, 
	status TEXT, 
	sub_total REAL, 
	total REAL, 
	total_tax REAL, 
	type TEXT, 
	updated_utc TIMESTAMP, 
	CONSTRAINT transactions_pkey PRIMARY KEY (_id)
)
Invoking: `sql_db_query` with `SELECT COUNT(*) AS total_transactions FROM transactions`


[(4,)]There are 4 transactions in the database.

> Finished chain.


{'input': 'how many transactions are there?',
 'output': 'There are 4 transactions in the database.'}

In [52]:
question = """Can you share the total expenses for a given account_id in a given date range. The output
should include account name and total expense"""

agent_executor.invoke(
    question
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


transactions, transactions__line_items, xero_accounts
Invoking: `sql_db_schema` with `{'table_names': 'transactions, transactions__line_items, xero_accounts'}`



CREATE TABLE transactions (
	_id TEXT NOT NULL, 
	_sdc_batched_at TIMESTAMP, 
	_sdc_extracted_at TIMESTAMP, 
	_sdc_received_at TIMESTAMP, 
	_sdc_sequence INTEGER, 
	_sdc_table_version INTEGER, 
	account_id TEXT, 
	bank_account_code TEXT, 
	bank_account_id TEXT, 
	bank_account_name TEXT, 
	bank_transaction_id TEXT, 
	contact_id TEXT, 
	contact_name TEXT, 
	currency_code TEXT, 
	date TIMESTAMP, 
	is_reconciled BOOLEAN, 
	line_amount_types TEXT, 
	reference TEXT, 
	status TEXT, 
	sub_total REAL, 
	total REAL, 
	total_tax REAL, 
	type TEXT, 
	updated_utc TIMESTAMP, 
	CONSTRAINT transactions_pkey PRIMARY KEY (_id)
)


CREATE TABLE transactions__line_items (
	_sdc_batched_at NUMERIC, 
	_sdc_level_0_id INTEGER NOT NULL, 
	_sdc_received_at NUMERIC,

{'input': 'Can you share the total expenses for a given account_id in a given date range. The output\nshould include account name and total expense',
 'output': 'I have executed the query, but it returned an empty result. This could be due to the specific account_id or date range not existing in the database. Please provide me with a valid account_id and date range so I can rerun the query and retrieve the total expenses for that account within the specified date range.'}

In [61]:
print("""
      I have executed the query, but it returned an empty result. 
      This could be due to the specific account_id or date range not existing in the database.
      Please provide me with a valid account_id and date range so I can rerun the query 
      and retrieve the total expenses for that account within the specified date range.
      """)


      I have executed the query, but it returned an empty result. 
      This could be due to the specific account_id or date range not existing in the database.
      Please provide me with a valid account_id and date range so I can rerun the query 
      and retrieve the total expenses for that account within the specified date range.
      
